# Annotator - Anomalies in time series

## Annotation

### Import the classes 

In [ ]:
from odin.annotator import AnomalyAnnotator, MetaPropertiesType
from odin.classes.timeseries.annotations_agreement import AnnotationAgreement

from odin.classes import TaskType

from odin.classes.timeseries import StandardScaler
from odin.classes.timeseries.anomaly_definition_strategies import AnomalyDefinitionStrategyTSAE

from odin.classes.timeseries import DatasetTSAnomalyDetection, TimeSeriesType, TSProposalsType, AnalyzerTSAnomalyDetection

### Define the relevant dataset paths

In [ ]:
GT_PATH = "../../test-data/anomaly_detection/gt.csv"
PROPOSALS_PATH = [('LSTM', "../../test-data/anomaly_detection/predictions.csv", TSProposalsType.REGRESSION)]
anomalies_path = '../../test-data/anomaly_detection/extended_anomalies.json'

### Define the dataset 

In [ ]:
my_dataset = DatasetTSAnomalyDetection(GT_PATH,
                                       TimeSeriesType.UNIVARIATE,
                                       anomalies_path=anomalies_path,
                                       proposals_paths=PROPOSALS_PATH, 
                                       index_gt='timestamp', 
                                       index_proposals='timestamp',
                                       scaler=None
                                      )

### Define the relevant annotation paths 

In [ ]:
anomalies_path = "../../test-data/anomaly_detection/userA_anomalies.json"
metaproperties_path = "../../test-data/anomaly_detection/userA_metaproperties.csv"
task_type = TaskType.TS_ANOMALY_DETECTION
output_name = "AnomalyDetectionExample" # name of the output anomalies file, that will be saved in the same path as anomalies_path

### Define the properties to annotate, their types and their values 

In [ ]:
properties = {"Duration3": (MetaPropertiesType.UNIQUE, ["Long", "Medium", "Short"]),
              "Type": (MetaPropertiesType.UNIQUE, ["Unknown", "Sudden spike"]),
              # This are momentarely deactivated until we provide support with the analyzer
              #"Medium": (MetaPropertiesType.COMPOUND, ["Paper", "Wood", "Canvas", "Stone", "Clay"]),
             #"Painter": (MetaPropertiesType.TEXT,),
             #"Characters":(MetaPropertiesType.CONTINUE, [0, 30])
             }

### Define the annotator 

In [ ]:
ds_generator = AnomalyAnnotator(dataset = my_dataset,
                                task_type=task_type,
                                anomaly_types=[],
                                anomaly_path=anomalies_path,
                                properties_and_values = properties,
                                ds_name=output_name,
                                metaproperties_path = metaproperties_path)

### Annotate the data 

In [ ]:
ds_generator.start_annotation("value")

## Inter-annotator agreement 
### Define the annotators' files

In [ ]:
agreement = AnnotationAgreement(my_dataset, "timestamp", "value", 
                                [("../../test-data/anomaly_detection/userA_anomalies.json", "A1"),
                                ("../../test-data/anomaly_detection/userB_anomalies.json", "A2"),
                                ("../../test-data/anomaly_detection/userC_anomalies.json", "A3")])

### Show the number of annotated anomalies per annotator

In [ ]:
agreement.show_anomalies_annotated()

### Compute the agreement between couples of annotators 

In [ ]:
v = agreement.calculate_iou_agreement()

### Display congruent anomalies (i.e., the ones on which there is agreement between the annotators) 

In [ ]:
anomalies = agreement.get_congruent_anomalies(agreement_threshold=20)
anomalies

### Export the annotated anomalies for which there is an agreement 

In [ ]:
anomalies.to_csv("./anomalies_recap.csv", index=False)